In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

data = pd.read_csv('/datasets/data.csv')
data['education'] = data['education'].str.lower() 
data.info()

#data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Data Preprocessing.

### Handling gaps in table data.

In [2]:
data['days_employed'].isna().sum()
data['total_income'].isna().sum()


data_whith_none = data.loc[(data['days_employed'] == '') & (data['total_income'] == '')].count()

data = data.drop_duplicates()
data.groupby('income_type')['total_income'].describe()

data['days_employed'] = data['days_employed'].fillna(value = 0)
data['total_income'] = data['total_income'].fillna(value = 0)

data['family_status_id'].describe()
data['family_status_id'].value_counts()

data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
data['dob_years'] = data['dob_years'].replace(0, data['dob_years'].mean())


def days_employed_fill (row):
    income_type_row = row['income_type']
    days_employed_row = row['days_employed']
    if income_type_row == 'пенсионер' and days_employed_row == 0:
        return 365213.306266
    elif income_type_row == 'студент' and days_employed_row == 0:
        return -578.751554
    elif income_type_row == 'безработный' and days_employed_row == 0:
        return 366413.652744
    elif income_type_row == 'в декрете' and days_employed_row == 0:
        return -3296.759962
    elif income_type_row == 'госслужащий' and days_employed_row == 0:
        return -2689.368353
    elif income_type_row == 'компаньон' and days_employed_row == 0:
        return 172357.950966
    elif income_type_row == 'предприниматель' and days_employed_row == 0:
        return -520.848083
    elif income_type_row == 'сотрудник' and days_employed_row == 0:
        return -1574.202821
    else:
        return days_employed_row

data['days_employed'] = data.apply(days_employed_fill ,axis = 1)

def total_income_fill (row):
    income_type_row = row['income_type']
    total_income_row = row['total_income']
    if income_type_row == 'пенсионер' and total_income_row == 0:
        return 118514.486412
    elif income_type_row == 'студент' and total_income_row == 0:
        return 98201.625314
    elif income_type_row == 'безработный' and total_income_row == 0:
        return 131339.751676
    elif income_type_row == 'в декрете' and total_income_row == 0:
        return 53829.130729
    elif income_type_row == 'госслужащий' and total_income_row == 0:
        return 150447.935283
    elif income_type_row == 'компаньон' and total_income_row == 0:
        return 172357.950966
    elif income_type_row == 'предприниматель' and total_income_row == 0:
        return 499163.144947
    elif income_type_row == 'сотрудник' and total_income_row == 0:
        return 142594.396847
    else:
        return total_income_row

data['total_income'] = data.apply(total_income_fill, axis = 1)


#print(2174 / 21525 *100)

data['gender'].max()

data['gender'].loc[data['gender'] == 'XNA'].count()



#data['days_employed'].describe()

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


1

### Data type replacement

In [3]:
data['total_income'] = data['total_income'].astype(int)
data['days_employed'] = data['days_employed'].astype(int)
data['dob_years'] = data['dob_years'].astype(int)


data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Обработка дубликатов

In [4]:
data.duplicated().sum()

0

In [5]:
def text_for_lemmatize(row):
    return m.lemmatize(row['purpose'])  

data['lemmas'] = data.apply(text_for_lemmatize, axis=1) 

In [6]:
def purpose_category(row):   
    lemmas = row['lemmas']
    if 'жилье' in lemmas or 'недвижимость' in lemmas or 'квартира' in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    
data['purpose_category'] = data.apply(purpose_category, axis = 1)

def total_income_category(row):
    row = row['total_income']
    if row < 145000:
        return 'ниже среднего'
    elif 145001 < row < 300000:
        return 'средний'
    else:
        return 'выше среднего'
    
data['total_income_category'] = data.apply(total_income_category, axis = 1)  


def children_category (row):  
    row = row['children']
    if row == 0:
        return 'без детей'
    elif 1 <= row <= 2:
        return 'семья с детьми'
    elif 3 <= row < 6:
        return 'многодетные'
    else:
        return 'over многодетные'
    
data['children_category'] = data.apply(children_category, axis = 1) 



In [7]:
data['total_income_category'].value_counts()

ниже среднего    11130
средний           8841
выше среднего     1483
Name: total_income_category, dtype: int64

### Шаг 3. Ответьте на вопросы

In [8]:
data_pivot_children = data.pivot_table(index = ['children_category'], columns='debt', values= 'total_income', aggfunc= 'count')

data_pivot_children['percentage_return'] = data_pivot_children[0] / (data_pivot_children[0] + data_pivot_children[1]) * 100

print(data_pivot_children)

debt                   0     1  percentage_return
children_category                                
без детей          13028  1063          92.456178
многодетные          349    31          91.842105
семья с детьми      6336   647          90.734641


### Вывод

In [9]:
data_family_status = data.pivot_table(index = ['family_status_id', 'family_status'], columns = 'debt', values= 'total_income', aggfunc='count')

data_family_status['percentage_return'] = data_family_status[0] / (data_family_status[0] + data_family_status[1]) * 100

data_family_status

,debt,0,1,percentage_return
family_status_id,family_status,,,
0,женат / замужем,11408,931,92.454818
1,гражданский брак,3763,388,90.652855
2,вдовец / вдова,896,63,93.430657
3,в разводе,1110,85,92.887029
4,Не женат / не замужем,2536,274,90.249110


### Вывод

In [10]:
data_total_income_category = data.pivot_table(index=['total_income_category'], columns='debt', values='total_income', aggfunc='count')

data_total_income_category['percentage_return'] = data_total_income_category[0] / (data_total_income_category[0] + data_total_income_category[1]) * 100
data_total_income_category.sort_values('percentage_return', ascending= False)

debt,0,1,percentage_return
total_income_category,,,
выше среднего,1377,106,92.852326
средний,8143,698,92.104966
ниже среднего,10193,937,91.581312


### Вывод